# Predict whether a customer will enroll for a certificate of deposit (CD)

This demo predict whether a customer will enroll for a certificate of deposit (CD)

The data has been labeled for your convenience and a column in the dataset identifies whether the customer is enrolled for a product offered by the bank. A version of this dataset is publicly available from the ML repository curated by the University of California, Irvine. This tutorial implements a supervised machine learning model since the data is labeled.

## Prepare the data

In [11]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")


Success - the MySageMakerInstance is in the us-east-1 region. You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [12]:
bucket_name = 'sagemaker-us-east-1-710299592439' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')

In [15]:
# Download dataset
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
  print(model_data[:10])
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.
   age  campaign  pdays  previous  no_previous_contact  not_working  \
0   56         1    999         0                    1            0   
1   57         1    999         0                    1            0   
2   37         1    999         0                    1            0   
3   40         1    999         0                    1            0   
4   56         1    999         0                    1            0   
5   45         1    999         0                    1            0   
6   59         1    999         0                    1            0   
7   41         1    999         0                    1            0   
8   24         1    999         0                    1            0   
9   25         1    999         0                    1            0   

   job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  ...  \
0           0                0                 0              1  ...   
1

## Shuffle the data and split into training and test data

In [16]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


## Train the model

To use an Amazon SageMaker pre-built XGBoost model, you will need to reformat the header and first column of the training data and load the data from the S3 bucket. 

In [20]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [21]:
# create the XGboost model
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[my_region],
                                    role, train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket_name, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)


In [22]:
# With the data loaded and the XGBoost estimator set up, 
# train the model using gradient optimization 

xgb.fit({'train': s3_input_train})

2020-07-01 10:30:10 Starting - Starting the training job...
2020-07-01 10:30:13 Starting - Launching requested ML instances.........
2020-07-01 10:31:57 Starting - Preparing the instances for training......
2020-07-01 10:33:04 Downloading - Downloading input data...
2020-07-01 10:33:27 Training - Downloading the training image..Arguments: train
[2020-07-01:10:33:48:INFO] Running standalone xgboost training.
[2020-07-01:10:33:48:INFO] Path /opt/ml/input/data/validation does not exist!
[2020-07-01:10:33:48:INFO] File size need to be processed in the node: 3.38mb. Available memory size in the node: 8506.04mb
[2020-07-01:10:33:48:INFO] Determined delimiter of CSV input is ','
[10:33:48] S3DistributionType set as FullyReplicated
[10:33:48] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[10:33:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 14 pruned nodes, max_depth=5
[0]#011train-error:0.100482
[1

# Deploy the model

In [23]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

---------------!

## predict whether customers in the test data enrolled for the bank product or not

In [24]:
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


# Evaluate model performance

In [25]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", 
        tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", 
        fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10785)    35% (151)
Purchase        10% (1143)     65% (278) 



# Cleanup

In [26]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'E38F526EA9F1E76E',
   'HostId': '/apeQC7odug04b0Kl7k9js+RWdiIUl3ENlIXbaFh26gNxI3V5Ho/uwT1/Ya6/cgXLfR2orJUago=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '/apeQC7odug04b0Kl7k9js+RWdiIUl3ENlIXbaFh26gNxI3V5Ho/uwT1/Ya6/cgXLfR2orJUago=',
    'x-amz-request-id': 'E38F526EA9F1E76E',
    'date': 'Wed, 01 Jul 2020 11:00:41 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'tko-ts-workshop-new-features/output/deepar-stores-sales-demo-2020-01-13-16-51-42-459/output/model.tar.gz'},
   {'Key': 'tko-ts-workshop-new-features/data/train/train_new_features.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2020-07-01-10-30-10-450/output/model.tar.gz'},
   {'Key': 'tko-ts-workshop/output/deepar-stores-sales-demo-2020-01-13-16-28-03-375/output/model.tar.gz'},
   {'Key': 'tko-ts-workshop/data/train/train.json'},
